In [2]:
from mesa import Agent, Model
from mesa.time import BaseScheduler
import numpy as np
import collections
from collections import defaultdict
import pandas as pd

In [3]:
class DataCollector:
    """ Class for collecting data generated by a Mesa model.
    A DataCollector is instantiated with dictionaries of names of model- and
    agent-level variables to collect, associated with attribute names or
    functions which actually collect them. When the collect(...) method is
    called, it collects these attributes and executes these functions one by
    one and stores the results.
    """

    model = None

    def __init__(self, model_reporters=None, agent_reporters=None, tables=None):
        """ Instantiate a DataCollector with lists of model and agent reporters.
        Both model_reporters and agent_reporters accept a dictionary mapping a
        variable name to either an attribute name, or a method.
        For example, if there was only one model-level reporter for number of
        agents, it might look like:
            {"agent_count": lambda m: m.schedule.get_agent_count() }
        If there was only one agent-level reporter (e.g. the agent's energy),
        it might look like this:
            {"energy": lambda a: a.energy}
        or like this:
            {"energy": "energy"}
        The tables arg accepts a dictionary mapping names of tables to lists of
        columns. For example, if we want to allow agents to write their age
        when they are destroyed (to keep track of lifespans), it might look
        like:
            {"Lifespan": ["unique_id", "age"]}
        Args:
            model_reporters: Dictionary of reporter names and attributes/funcs
            agent_reporters: Dictionary of reporter names and attributes/funcs.
            tables: Dictionary of table names to lists of column names.
        """
        self.model_reporters = {}
        self.agent_reporters = {}

        self.model_vars = {}
        self.agent_vars = {}
        self.tables = {}

        if model_reporters is not None:
            for name, reporter in model_reporters.items():
                self._new_model_reporter(name, reporter)

        if agent_reporters is not None:
            for name, reporter in agent_reporters.items():
                self._new_agent_reporter(name, reporter)

        if tables is not None:
            for name, columns in tables.items():
                self._new_table(name, columns)

    def _new_model_reporter(self, name, reporter):
        """ Add a new model-level reporter to collect.
        Args:
            name: Name of the model-level variable to collect.
            reporter: Attribute string, or function object that returns the
                      variable when given a model instance.
        """
        if type(reporter) is str:
            reporter = self._make_attribute_collector(reporter)
        self.model_reporters[name] = reporter
        self.model_vars[name] = []

    def _new_agent_reporter(self, name, reporter):
        """ Add a new agent-level reporter to collect.
        Args:
            name: Name of the agent-level variable to collect.
            reporter: Attribute string, or function object that returns the
                      variable when given a model instance.
        """
        if type(reporter) is str:
            reporter = self._make_attribute_collector(reporter)
        self.agent_reporters[name] = reporter
        self.agent_vars[name] = []

    def _new_table(self, table_name, table_columns):
        """ Add a new table that objects can write to.
        Args:
            table_name: Name of the new table.
            table_columns: List of columns to add to the table.
        """
        new_table = {column: [] for column in table_columns}
        self.tables[table_name] = new_table

    def collect(self, model):
        """ Collect all the data for the given model object. """
        if self.model_reporters:
            for var, reporter in self.model_reporters.items():
                self.model_vars[var].append(reporter(model))

        if self.agent_reporters:
            for var, reporter in self.agent_reporters.items():
                agent_records = []
                for id, agent in model.schedule.agents_by_type['Customer'].items():
                    agent_records.append((id, reporter(agent)))
                self.agent_vars[var].append(agent_records)

    def add_table_row(self, table_name, row, ignore_missing=False):
        """ Add a row dictionary to a specific table.
        Args:
            table_name: Name of the table to append a row to.
            row: A dictionary of the form {column_name: value...}
            ignore_missing: If True, fill any missing columns with Nones;
                            if False, throw an error if any columns are missing
        """
        if table_name not in self.tables:
            raise Exception("Table does not exist.")

        for column in self.tables[table_name]:
            if column in row:
                self.tables[table_name][column].append(row[column])
            elif ignore_missing:
                self.tables[table_name][column].append(None)
            else:
                raise Exception("Could not insert row with missing column")

    @staticmethod
    def _make_attribute_collector(attr):
        '''
        Create a function which collects the value of a named attribute
        '''

        def attr_collector(obj):
            return getattr(obj, attr)

        return attr_collector

    def get_model_vars_dataframe(self):
        """ Create a pandas DataFrame from the model variables.
        The DataFrame has one column for each model variable, and the index is
        (implicitly) the model tick.
        """
        return pd.DataFrame(self.model_vars)

    def get_agent_vars_dataframe(self):
        """ Create a pandas DataFrame from the agent variables.
        The DataFrame has one column for each variable, with two additional
        columns for tick and agent_id.
        """
        data = defaultdict(dict)
        for var, records in self.agent_vars.items():
            for step, entries in enumerate(records):
                for entry in entries:
                    agent_id = entry[0]
                    val = entry[1]
                    data[(step, agent_id)][var] = val
        df = pd.DataFrame.from_dict(data, orient="index")
        #df.index.names = ["Step", "AgentID"]
        return df

    def get_table_dataframe(self, table_name):
        """ Create a pandas DataFrame from a particular table.
        Args:
            table_name: The name of the table to convert.
        """
        if table_name not in self.tables:
            raise Exception("No such table.")
        return pd.DataFrame(self.tables[table_name])

In [207]:
import random
from collections import defaultdict

from mesa.time import RandomActivation


class ActivationByType(RandomActivation):
    """A scheduler which activates each type of agent once per step, in random
    order, with the order reshuffled every step.
    Assumes that all agents have a step() method.
    
    """

    def __init__(self, model):
        super().__init__(model)
        self.agents_by_type = defaultdict(dict)

    def add(self, agent):
        """Add an Agent object to the schedule
        
        Args:
            agent: An Agent to be added to the schedule.
        
        """

        self._agents[agent.unique_id] = agent
        agent_type = agent.__class__.__name__
        if agent_type in ['Investor', 
                          'Contributor', 
                          'VerificationSponsor',
                          'CharitableSponsor']: agent_type = 'Customer'
        self.agents_by_type[agent_type][agent.unique_id] = agent

    def remove(self, agent):
        """Remove all instances of a given agent from the schedule.
        
        """

        del self._agents[agent.unique_id]

        agent_type = self.__class__.__name__
        del self.agents_by_type[agent_type][agent.unique_id]

    def step(self):
        """Executes the step of each agent type, one at a time, in random order.
        
        """
        for agent_type in ['Customer', 'Teo']:
            self.step_type(agent_type)
        self.steps += 1
        self.time += 1


    def step_type(self, type):
        """Run all agents of a given type.
        
        Args:
            type: Class name of the type to run.
            
        """
        for agent_key in list(self.agents_by_type[type].keys()):
            self.agents_by_type[type][agent_key].step()


In [229]:
class Teo(Agent):
    
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.action_register = []
        
    def register_deposit(self, agent, value):
        """Registers intended deposit value of an agent in the action-register.

        Args:
            agent (Agent): Agent instance.
            value (int): Intendend deposit value in euro.

        """
        action = {
            'unique_id': agent.unique_id,
            'action': 'deposit',
            'value': value
        }
        self.action_register.append(action)
        
    def execute_deposits(self):
        """Executes all deposit actions from the action-register.
      
        """
        deposits = [v for v in self.action_register if v['action'] == 'deposit']
        for deposit in deposits:
            model.schedule.agents_by_type['Customer'][deposit['unique_id']].euro_wallet += deposit['value']

    def register_sponsorship(self, agent, value):
        """Registers the intended sponsorship teos of an agent in the action-register
        if the agent is allowed to register the intended amount.

        Args:
            agent (Agent): Agent instance.
            value (int): Intendend sponsorship amount in teos.

        """
        if value <= agent.teo_wallet - agent.staged_teo:
            action = {
                'unique_id': agent.unique_id,
                'action': 'sponsorship',
                'value': value
            }
            self.action_register.append(action)
            agent.staged_teo += value       
            
    def execute_sponsorship(self):
        """Executes all sponsorship actions from the action-register.
      
        """
        sponsorships = [v for v in self.action_register if v['action'] == 'sponsorship']
        for sponsorship in sponsorships:
            model.schedule.agents_by_type['Customer'][sponsorship['unique_id']].staged_teo += sponsorship['value']
            
    def register_contribution(self, agent, value):
        """Registers the intended contribution hours of an agent in the action-register
        if the agent is allowed to register the intended amount.

        Args:
            agent (Agent): Agent instance.
            value (int): Intendend contribution hours.

        """
        if value <= agent.hour_wallet:
            action = {
                'unique_id': agent.unique_id,
                'action': 'contribution',
                'value': value
            }
            self.action_register.append(action)
            
    def execute_contribution(self):
        """Executes all contribution actions from the action-register.
        
        """
        contributions = [v for v in self.action_register if v['action'] == 'contribution']
        for contribution in contributions:
            model.schedule.agents_by_type['Customer'][contribution['unique_id']].hour_wallet -= contribution['value']
            model.schedule.agents_by_type['Customer'][contribution['unique_id']].contributed_hours += contribution['value']

    def register_withdraw(self, agent, value):
        """Registers the intended withdraw value of an agent in the action-register
        if the agent is allowed to register the intended amount.

        Args:
            agent (Agent): Agent instance.
            value (int): Intendend withdraw value in euros.

        """
        if value <= agent.euro_wallet - agent.staged_euro and model.schedule.steps - agent.last_withdraw_tick >= 2:
            action = {
                'unique_id': agent.unique_id,
                'action': 'withdraw',
                'value': value
            }
            self.action_register.append(action)
            agent.staged_euro += value
        
    def execute_withdraws(self):
        """Executes all withdraw actions from the action-register.
        
        """
        withdraws = [v for v in self.action_register if v['action'] == 'withdraw']
        for withdraw in withdraws:
            model.schedule.agents_by_type['Customer'][withdraw['unique_id']].euro_wallet -= withdraw['value']
            model.schedule.agents_by_type['Customer'][withdraw['unique_id']].last_withdraw_tick = model.schedule.steps

    def register_euro_exchange(self, agent, value):
        """Registers the intended euro->teo exchange value of an agent in the action-register
        if the agent is allowed to register the intended amount.

        Args:
            agent (Agent): Agent instance.
            value (int): Intendend exchange value in euros.

        """
        if value <= agent.euro_wallet - agent.staged_euro:
            action = {
                'unique_id': agent.unique_id,
                'action': 'euro_exchange',
                'value': value
            }
            self.action_register.append(action)
            agent.staged_euro += value
            
    def register_teo_exchange(self, agent, value):
        """Registers the intended teo->euro exchange value of an agent in the action-register
        if the agent is allowed to register the intended amount.

        Args:
            agent (Agent): Agent instance.
            value (int): Intendend exchange value in teos.

        """
        if value <= agent.teo_wallet - agent.staged_teo:
            action = {
                'unique_id': agent.unique_id,
                'action': 'teo_exchange',
                'value': value
            }
            self.action_register.append(action)
            agent.staged_teo += value
            
    def execute_exchanges(self):
        """Executes all exchanges from the action-register. 
        
        If the teo-exchange amount > euro-exchange amount, first all euro-to-teo exchanges are
        executed, then teo-to-euro exchanges are executed randomly until the exchanged amount exceeds
        the exchangeable euro amount. The last exchange is potentially done only partially. 
        
        """
        #TODO: make this nicer and less verbose!
        exchanged_euros = 0
        exchanged_teos = 0
        
        euro_exchanges = [v for v in self.action_register if v['action'] == 'euro_exchange']
        teo_exchanges = [v for v in self.action_register if v['action'] == 'teo_exchange']
    
        if len(teo_exchanges) == 0 or len(euro_exchanges) == 0 or \
        (sum([v['value'] for v in teo_exchanges]) == 0 and sum([v['value'] for v in euro_exchanges]) == 0):
            return
        else:
            teo_exchange_volume = sum([v['value'] for v in teo_exchanges])
            euro_exchange_volume = sum([v['value'] for v in euro_exchanges])
            #in the case that euro register is larger, fullfill euro exchanges in random order until teo amount reached
            if euro_exchange_volume >= teo_exchange_volume:
                # transfer money for all agents in teo register
                for exchange in teo_exchanges:
                    model.schedule.agents_by_type['Customer'][exchange['unique_id']].teo_wallet -= exchange['value']
                    model.schedule.agents_by_type['Customer'][exchange['unique_id']].euro_wallet += exchange['value']
                    model.schedule.agents_by_type['Customer'][exchange['unique_id']].exchanged_teos += exchange['value']
                # fullfill euro exchanges in random order until total teo amount is reached
                random.shuffle(euro_exchanges)
                for exchange in euro_exchanges:
                    if exchanged_euros + exchange['value'] > teo_exchange_volume:
                        remaining_euros = teo_exchange_volume - exchanged_euros
                        model.schedule.agents_by_type['Customer'][exchange['unique_id']].euro_wallet -= remaining_euros
                        model.schedule.agents_by_type['Customer'][exchange['unique_id']].teo_wallet += remaining_euros
                        model.schedule.agents_by_type['Customer'][exchange['unique_id']].exchanged_euros += remaining_euros
                        break
                    model.schedule.agents_by_type['Customer'][exchange['unique_id']].euro_wallet -= exchange['value']
                    model.schedule.agents_by_type['Customer'][exchange['unique_id']].teo_wallet += exchange['value']
                    model.schedule.agents_by_type['Customer'][exchange['unique_id']].exchanged_euros += exchange['value']
                    exchanged_euros += exchange['value']
            #in the case that teo register is larger, fullfill teo exchanges in random order until euro amount reached      
            else:
                # transfer money for all agents in euro register
                for exchange in euro_exchanges:
                    model.schedule.agents_by_type['Customer'][exchange['unique_id']].euro_wallet -= exchange['value']
                    model.schedule.agents_by_type['Customer'][exchange['unique_id']].teo_wallet += exchange['value']
                    model.schedule.agents_by_type['Customer'][exchange['unique_id']].exchanged_euros += exchange['value']
                # fullfill euro exchanges in random order until total teo amount is reached
                random.shuffle(teo_exchanges)
                for exchange in teo_exchanges:
                    if exchanged_teos + exchange['value'] > euro_exchange_volume:
                        remaining_teos = euro_exchange_volume - exchanged_teos
                        model.schedule.agents_by_type['Customer'][exchange['unique_id']].teo_wallet -= remaining_teos
                        model.schedule.agents_by_type['Customer'][exchange['unique_id']].euro_wallet += remaining_teos
                        model.schedule.agents_by_type['Customer'][exchange['unique_id']].exchanged_teos += remaining_teos

                        break
                    model.schedule.agents_by_type['Customer'][exchange['unique_id']].teo_wallet -= exchange['value']
                    model.schedule.agents_by_type['Customer'][exchange['unique_id']].euro_wallet += exchange['value']
                    model.schedule.agents_by_type['Customer'][exchange['unique_id']].exchanged_teos += exchange['value']

                    exchanged_teos += exchange['value']
    
    def reward_contributions(self):
        """Method that rewards agents that contributed in the current tick.

        """        
        contribution_reward_per_hour = get_contribution_reward_per_hour(model)
        #payout to agents according to the number of hours they contributed
        for agent_id, agent in model.schedule.agents_by_type['Customer'].items():
            agent.teo_wallet += agent.contributed_hours * contribution_reward_per_hour

    def reward_exchanges(self):
        """Method that rewards agents that exchanged euros for teos in the current tick.

        """
        exchange_reward_per_euro = get_exchange_reward_per_euro(model)
        #payout to agents according to the number of hours they contributed
        for agent_id, agent in model.schedule.agents_by_type['Customer'].items():
            agent.teo_wallet += agent.exchanged_euros * exchange_reward_per_euro

    
    def reset_parameters(self):
        self.action_register = []
    
    def step(self):
        self.execute_deposits()
        self.execute_contribution()
        self.execute_sponsorship()
        self.execute_exchanges()
        self.execute_withdraws()
        self.reward_contributions()
        self.reward_exchanges()
        self.reset_parameters()
        


In [232]:
def get_exchange_reward_per_euro(model):
    """Method that returns the reward per exchanged euro for the current tick.

    Args:
        model (Model): Instance of the model class.
        
    """
    exchanged_euros = get_exchanged_euros(model)
    if exchanged_euros == 0:
        return 0
    total_euros = get_total_euros(model) 
    total_teos = get_total_teos(model)
    exchange_pool = (total_euros - total_teos)*BUFFER_PERCENTAGE*EXCHANGE_REWARD_PERCENTAGE_OF_BUFFER
    exchange_reward_per_euro = exchange_pool / exchanged_euros 
    return exchange_reward_per_euro
    
def get_contribution_reward_per_hour(model):
    """Method that returns the reward per contributed hour for the current tick.

    Args:
        model (Model): Instance of the model class.

    """
    contributed_hours = get_total_hours(model)
    if contributed_hours == 0:
        return
    total_teos = get_total_teos(model)
    total_euros = get_total_euros(model)
    contribution_pool = (total_euros - total_teos)*(1-BUFFER_PERCENTAGE)
    contribution_reward_per_hour = contribution_pool / contributed_hours
    return contribution_reward_per_hour

def get_exchanged_euros(model):
    """Method that returns all euros that were exchanged in the current tick.

    Args:
        model (Model): Instance of the model class.

    """
    # list of amounts of all Customers' exchanged euros
    exchanged_euros = np.sum([v.exchanged_euros for k, v in model.schedule.agents_by_type['Customer'].items()])
    # return the sum of agents' exchanged euros
    return np.sum(exchanged_euros)

def get_total_teos(model):
    """Method that returns all teos in the system at the end of the current tick.

    Args:
        model (Model): Instance of the model class.

    """
    # list of amounts of all Customers' teo's
    total_teos = [v.teo_wallet for k, v in model.schedule.agents_by_type['Customer'].items()]
    # return the sum of agents' teos
    return np.sum(total_teos)

def get_total_euros(model):
    """Method that returns all euros in the system at the end of the current tick.

    Args:
        model (Model): Instance of the model class.

    """
    # list of amounts of all Customers' euro
    total_euros = [v.euro_wallet for k, v in model.schedule.agents_by_type['Customer'].items()]
    # return the sum of agents' euros
    return np.sum(total_euros)

def get_total_hours(model):
    """Method that returns all hours that were contributed in the current tick.

    Args:
        model (Model): Instance of the model class.

    """
    # list of amounts of all Customers' contributed hours in current month
    total_hours = [v.contributed_hours for k, v in model.schedule.agents_by_type['Customer'].items()]
    # return the sum of agents' hours
    return np.sum(total_hours)

In [255]:
class Customer(Agent):
    """Customer agent base class."""
    
    def __init__(self, unique_id, model, teo):
        super().__init__(unique_id, model)
        self.monthly_deposit = 0
        self.monthly_hours = 0
        
        self.euro_wallet = 0
        self.teo_wallet = 0
        
        self.deposit_intent = 0
        self.contribution_intent = 0
        self.sponsor_intent = 0
        self.teo_exchange_intent = 0
        self.euro_exchange_intent = 0
        self.withdraw_intent = 0
        
        self.hour_wallet = self.monthly_hours
        self.staged_euro = 0
        self.staged_teo = 0
        self.contributed_hours = 0
        self.exchanged_euros = 0
        self.exchanged_teos = 0
               
        self.teo = teo

        self.last_withdraw_tick = -1
    
         
    def register_deposit(self, deposit_intent):
        """Registers the intended deposit amount with TEO.

        Args:
            deposit_intent (int): Indended deposit euros.

        """
        if deposit_intent > 0:
            self.teo.register_deposit(self, deposit_intent)
        
    def register_sponsorship(self, sponsor_intent):
        """Registers the intended sponsorship amount with TEO.

        Args:
            sponsor_intent (int): Indended sponsoring teos.

        """
        if sponsor_intent > 0:
            self.teo.register_sponsorship(self, sponsor_intent)  
        
    def register_contribution(self, contribution_intent):
        """Registers the intended contribution amount with TEO.

        Args:
            contribution_intent (int): Indended contribution hours.

        """
        if contribution_intent > 0:
            self.teo.register_contribution(self, contribution_intent)  
                     
    def register_withdraw(self, withdraw_intent):
        """Registers the intended withdraw amount with TEO.

        Args:
            withdraw_intent (int): Indended withdraw euros.

        """        
        if withdraw_intent > 0:
            self.teo.register_withdraw(self, withdraw_intent)
        
    def register_euro_exchange(self, exchange_intent):
        """Registers the intended euro->teo exchange amount with TEO.

        Args:
            exchange_intent (int): Indended euros to exchange for teos.

        """
        if exchange_intent > 0:
            self.teo.register_euro_exchange(self, exchange_intent)  
        
    def register_teo_exchange(self, exchange_intent):
        """Registers the intended teo->euro exchange amount with TEO.

        Args:
            exchange_intent (int): Indended teos to exchange for euros.

        """        
        if exchange_intent > 0:
            self.teo.register_teo_exchange(self, exchange_intent)  
      
    def reset_parameters(self):
        """Resets temporary parameters at the beginning of a new tick.

        """       
        self.staged_teo = 0
        self.staged_euro = 0
        self.hour_wallet = self.monthly_hours
        self.contributed_hours = 0
        self.exchanged_euros = 0
        self.exchanged_teos = 0

    def step(self):
        """Step method defining the ordered action to be taken each step.

        """      
        self.reset_parameters()

        
        self.register_deposit(self.deposit_intent)
        self.register_contribution(self.contribution_intent)
        self.register_sponsorship(self.sponsor_intent)
        self.register_euro_exchange(self.euro_exchange_intent)
        self.register_teo_exchange(self.teo_exchange_intent)
        self.register_withdraw(self.withdraw_intent)
        
        

In [256]:
class VerificationSponsor(Customer):
    """Class describing the verfication sponsor. 
    
    A verification sponsor deposits 100 euro once and sponsors 100 teos each round. 
    She contributes occassionally and withdraws all rewards from the system.
    She has the following properties:
        - deposits 100 euro in first month
        - mininum coins in system: 100
        - contributes 2 hours randomly in 1 of 3 months 
        - exchanges teo->euro exceeding 100 coins (the rewards)
        - withdraws everything from euro-wallet
    """

    def __init__(self, unique_id, model, teo):
        super().__init__(unique_id, model, teo)
        self.monthly_deposit = 100
        self.monthly_hours = 2
            
    
    def step(self):
        self.reset_parameters()

        self.sponsor_intent = min([self.teo_wallet, self.monthly_deposit]) #always sponsor max 100
        self.teo_exchange_intent = max([self.teo_wallet - self.sponsor_intent, 0]) #exchange non-sponsored teos back
        self.euro_exchange_intent = max([self.monthly_deposit - self.teo_wallet, 0]) #only exchange euros if teos < 100
        self.withdraw_intent = max([self.teo_wallet + self.euro_wallet - self.monthly_deposit, 0]) #always withdraw rewards
               
        if model.schedule.steps == 0:
            self.deposit_intent = self.monthly_deposit
            self.register_deposit(self.deposit_intent)
        if np.random.uniform(0, 1) < 1/3:
            self.contribution_intent = self.monthly_hours
            self.register_contribution(self.contribution_intent)
        
        self.register_sponsorship(self.sponsor_intent) 
        self.register_euro_exchange(self.euro_exchange_intent) 
        self.register_teo_exchange(self.teo_exchange_intent)
        self.register_withdraw(self.withdraw_intent)
        

class CharitableSponsor(Customer):
    """Class describing the charitable sponsor.
    
    A charitable sponsor leaves all money within the system and has 
    the following properties:    
        - deposits every month
        - exchanges always all euros to teo
        - sponsors all teos
        - does not exchange teo->euro and does not withdraw
    """

    def __init__(self, unique_id, model, teo):
        super().__init__(unique_id, model, teo)
        self.monthly_deposit = 50
        self.monthly_hours = 0

    def step(self):
        self.reset_parameters()

        self.deposit_intent = self.monthly_deposit
        self.sponsor_intent = self.teo_wallet #sponsor all teos
        self.euro_exchange_intent = self.euro_wallet #always exchange all euros->teos
        
        self.register_deposit(self.deposit_intent)
        self.register_contribution(self.contribution_intent)
        self.register_sponsorship(self.sponsor_intent) 
        self.register_euro_exchange(self.euro_exchange_intent) 
        self.register_teo_exchange(self.teo_exchange_intent)
        self.register_withdraw(self.withdraw_intent)
        
class Contributor(Customer):
    """Class describing the contributor agent.

    A contributor that has the following properties:
        - Has a monthly deposit of 10
        - Has 80 monthly hours
        - Contributes all monthly hours
        - Always exchanges all teos for euros
        - Always withdraws all euros from euro-wallet

    """
    
    def __init__(self, unique_id, model, teo):
        super().__init__(unique_id, model, teo)
        self.monthly_deposit = 10
        self.monthly_hours = 80
    
    def step(self):
        self.reset_parameters()

        self.deposit_intent = self.monthly_deposit
        self.contribution_intent = self.monthly_hours
        self.teo_exchange_intent = self.teo_wallet #always exchanges all teos to euros
        self.withdraw_intent = self.euro_wallet
        
        self.register_deposit(self.deposit_intent)
        self.register_contribution(self.contribution_intent)
        self.register_sponsorship(self.sponsor_intent) 
        self.register_euro_exchange(self.euro_exchange_intent) 
        self.register_teo_exchange(self.teo_exchange_intent)
        self.register_withdraw(self.withdraw_intent)
        

class Investor(Customer):
    """Class describing the investor agent.

    An investor that has the following properties:
    - Has a monthly deposit of 400
    - Always exchanges all teos for euros
    - Always withdraws all euros for teos
    - Randomly withdraws at a point between 12 and 24 months/ticks after last withdraw/start (uniform)
    - The withdraw amount is random between 20-80% (uniform)
    """

    def __init__(self, unique_id, model, teo):
        super().__init__(unique_id, model, teo)
        self.monthly_deposit = 400
        self.monthly_hours = 0
    
    def step(self):
        self.reset_parameters()

        self.deposit_intent = self.monthly_deposit
        self.teo_exchange_intent = self.teo_wallet #always exchanges all teos to euros
        self.euro_exchange_intent = self.euro_wallet #always exchanges all euros to teos
        
        self.register_deposit(self.deposit_intent)
        self.register_contribution(self.contribution_intent)
        self.register_sponsorship(self.sponsor_intent) 
        self.register_euro_exchange(self.euro_exchange_intent) 
        self.register_teo_exchange(self.teo_exchange_intent)
        self.register_withdraw(self.withdraw_intent)

In [260]:
class TeoModel(Model):
    """A model simulating the TEO mechanics.
    """

    def __init__(self, n_contributors, n_char_sponsors, n_ver_sponsors, n_investors):
        """Initializes a new TEO model with a certain number of agents of each type.
               
        Args:
            n_contributors (int): Number of contributors.
            n_char_sponsors (int): Number of charitable sponsors.
            n_ver_sponsors (int): Number of verification sponsors.
            n_investors (int): Number of contributors.

        """
        self.step_id = 0
        self.n_contributors = n_contributors
        self.n_sponsors = n_sponsors
        self.n_investors = n_investors
        self.schedule = ActivationByType(self)
        self.datacollector = DataCollector(model_reporters={
                                              "Total Euros": get_total_euros,
                                              "Total Teos": get_total_teos,
                                              "Contributed Hours": get_total_hours,
                                              "Reward per Contrib Hour": get_contribution_reward_per_hour,
                                              "Reward per Exchanged Euro": get_exchange_reward_per_euro
                                          },
                                          agent_reporters={
                                              "Euro Wallet": lambda a: a.euro_wallet,
                                              "Teo Wallet": lambda a: a.teo_wallet,     
                                              "Registered Sponsored Teos": lambda a: a.sponsor_intent,
                                              "Registered Exchange Teos": lambda a: a.teo_exchange_intent,
                                              "Exchanged Teos": lambda a: a.exchanged_teos,
                                              "Registered Exchange Euros": lambda a: a.euro_exchange_intent,                                              
                                              "Exchanged Euros": lambda a: a.exchanged_euros,                                              
                                              "Contributed Hours": lambda a: a.contributed_hours,
                                              "Last withdraw tick": lambda a: a.last_withdraw_tick
                                              
                                          })

        # Create agents
        teo = Teo(0, self)
        self.schedule.add(teo)
        
        """
        Contributor: 60%
        Investors: 25%
        VerficationSponsors: 10%
        CharitySponsors: 5%
        
        New Users: 10% (Reduces to 5% over 24 months)
        User Churn: 5%
        
        """
        
        for i in range(self.n_contributors):
            a = Contributor('Contributor_'+str(i), self, teo)
            self.schedule.add(a)
            
        for i in range(self.n_ver_sponsors):
            a = VerificationSponsor('Ver_Sponsor_'+str(i), self, teo)
            self.schedule.add(a)
            
        for i in range(self.n_char_sponsors):
            a = CharitableSponsor('Char_Sponsor_'+str(i), self, teo)
            self.schedule.add(a)
            
        for i in range(self.n_investors):
            a = Investor('Investor_'+str(i), self, teo)
            self.schedule.add(a)
            
        

        self.running = True

    def step(self):
        self.schedule.step()
        # collect data
        self.datacollector.collect(self)


In [261]:
model.schedule.steps

1

In [262]:
BUFFER_PERCENTAGE = 0.2
EXCHANGE_REWARD_PERCENTAGE_OF_BUFFER = 0.2

model = TeoModel(2, 1, 1)

for i in range(10):
    model.step()
    
results = model.datacollector.get_agent_vars_dataframe().reset_index()
results = results.rename(columns={'level_0': 'tick', 'level_1': 'agent_id'})

In [263]:
results

,tick,agent_id,Euro Wallet,Teo Wallet,Registered Sponsored Teos,Registered Exchange Teos,Exchanged Teos,Registered Exchange Euros,Exchanged Euros,Contributed Hours,Last withdraw tick
0,0,Contributor_0,10.000000,208.000000,0.000000,0.000000,0.000000,0.000000,0.000000,80,-1
1,0,Contributor_1,10.000000,208.000000,0.000000,0.000000,0.000000,0.000000,0.000000,80,-1
2,0,Investor_0,400.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,-1
3,0,VerificationSponsor_0,100.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0,-1
4,1,Contributor_0,218.000000,199.111111,0.000000,208.000000,208.000000,0.000000,0.000000,80,1
5,1,Contributor_1,218.000000,199.111111,0.000000,208.000000,208.000000,0.000000,0.000000,80,1
6,1,Investor_0,400.000000,403.876923,0.000000,0.000000,0.000000,400.000000,400.000000,0,-1
7,1,VerificationSponsor_0,84.000000,21.132855,0.000000,0.000000,0.000000,100.000000,16.000000,2,-1
8,2,Contributor_0,427.111111,202.127465,0.000000,199.111111,199.111111,0.000000,0.000000,80,1
9,2,Contributor_1,228.000000,401.238576,0.000000,199.111111,0.000000,0.000000,0.000000,80,1


In [264]:
results[results.agent_id == 'VerificationSponsor_0']

,tick,agent_id,Euro Wallet,Teo Wallet,Registered Sponsored Teos,Registered Exchange Teos,Exchanged Teos,Registered Exchange Euros,Exchanged Euros,Contributed Hours,Last withdraw tick
3,0,VerificationSponsor_0,100.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0,-1
7,1,VerificationSponsor_0,84.000000,21.132855,0.000000,0.000000,0.000000,100.000000,16.000000,2,-1
11,2,VerificationSponsor_0,0.000000,105.727298,21.132855,0.000000,0.000000,78.867145,78.867145,2,2
15,3,VerificationSponsor_0,0.000000,104.375424,100.000000,5.727298,0.000000,0.000000,0.000000,2,2
19,4,VerificationSponsor_0,0.000000,108.264013,100.000000,4.375424,0.000000,0.000000,0.000000,2,2
23,5,VerificationSponsor_0,8.264013,100.000000,100.000000,8.264013,8.264013,0.000000,0.000000,0,2
27,6,VerificationSponsor_0,0.000000,100.000000,100.000000,0.000000,0.000000,0.000000,0.000000,0,6
31,7,VerificationSponsor_0,0.000000,102.331515,100.000000,0.000000,0.000000,0.000000,0.000000,2,6
35,8,VerificationSponsor_0,2.331515,100.000000,100.000000,2.331515,2.331515,0.000000,0.000000,0,6
39,9,VerificationSponsor_0,0.000000,100.000000,100.000000,0.000000,0.000000,0.000000,0.000000,0,9


In [265]:
model.datacollector.get_model_vars_dataframe()

,Contributed Hours,Reward per Contrib Hour,Reward per Exchanged Euro,Total Euros,Total Teos
0,160,0.520000,0.000000,520.000000,416.000000
1,162,0.477867,0.009305,920.000000,823.232000
2,162,0.485106,0.008206,1334.867145,1236.633197
3,162,-0.129780,-0.001546,1099.756034,1126.036450
4,162,0.373304,0.004456,1519.756034,1444.161874
5,160,-0.089949,-0.000773,1350.464922,1368.454737
6,160,0.377996,0.002592,1762.200909,1686.601644
7,162,0.223825,0.001207,1922.667539,1877.342887
8,160,0.446712,0.002199,2342.667539,2253.325206
9,160,0.148958,0.000599,2408.490171,2378.698498
